In [2]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import time
import datetime as dt
import itertools
import re
import math
from collections import Counter
import operator

#plot
import matplotlib.pyplot as plt
import seaborn as sns

!pip install --upgrade scipy

In [3]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import preprocessing_Origins,is_day,vertical_travel_distance, li_missingZone_mvtPerc_DU, li_event_chaoticmvt_z_d,\
ZoneVariable, HenDailyVariable_Origins, boxplot_distribution_entropy,\
nbr_transition, max_duration_zones, openDevice, kmeans_clustering, sampen
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


2


# Define parameters

In [4]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
dico_matching = config.dico_matching
print(id_run)
path_extracted_data_visual = os.path.join(path_extracted_data,'visual')
#create a director if not existing
if not os.path.exists(path_extracted_data_visual):
    os.makedirs(path_extracted_data_visual)

change the configuration file if not done yet!
correctlightschedule_


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Variables at Hen level

In [5]:
#WARNING: we do it at seconds level, so the miliseconds from the general cleaning is not used!! 

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_CLEANEDDATA.csv'), sep=';', parse_dates=['Timestamp', 'date']) 
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2705760, 57)


,Unnamed: 0,Timestamp,HenID,Zone,PenID,ts_order,TagID,signalstrength,system,time,...,previous_duration_bounded_mn,nbr_nan,model_prediction,next_record_date,previous_record_date,next_zone,previous_zone,previous_previous_zone,correction_is_consecutive_equal_initial_zone,is_WG_open
0,0,2020-09-29 09:07:00,hen_71,3_Zone,pen12,0,tag_162,10,10 - 12,09:07:00,...,0.0,1,1.0,2020-09-29 09:08:26,NaN,3_Zone,NaN,NaN,False,False
1,1,2020-09-29 09:08:12,hen_7,3_Zone,pen11,1,tag_140,10,10 - 12,09:08:12,...,0.0,1,1.0,2020-09-29 09:12:16,NaN,3_Zone,NaN,NaN,False,False
2,13,2020-09-29 09:19:19,hen_10,3_Zone,pen11,13,tag_153,14,10 - 12,09:19:19,...,0.0,1,1.0,2020-09-29 09:20:27,NaN,3_Zone,NaN,NaN,False,False


In [7]:
#open time series per session and compute the variables for each session (car time series make sence at session level), et en 
#plus des variables tel que running entropy over the whole session ateach last timestamp of each level make sence only at 
#session level
#once to much data, do this per pen!!! with name=pens!!
START_TIME = time.perf_counter()
for p in tqdm.tqdm(df['PenID'].unique()):
    print(p)
    #&(df['date']<=dt.datetime(2020,11,10))
    df_daily = HenDailyVariable_Origins(df[(df['PenID']==p)], config, name_=p, timestamp_name='Timestamp', 
                                        has_cons_equal_zone=False)
    print(df_daily.shape)
    df_daily.head(3)
    print(df_daily['level'].min())
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))  

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

pen12


C:\Users\camil\Desktop\animals_code\PhD-AnimalWelfare\UTILS.py:2748: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values([timestamp_name], inplace=True)
C:\Users\camil\Desktop\animals_code\PhD-AnimalWelfare\UTILS.py:2750: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[timestamp_name] = df[timestamp_name].map(lambda x: dt.datetime(x.year,x.month,x.day,x.hour,x.minute, x.second))


----------------- Create time serie
in this time series there is 31 hens
The initial starting date in over all is: 2020-09-29 09:07:00, and the ending date will be: 2021-07-25 17:08:42
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-09-29 09:07:00, and the ending date will be: 2021-07-25 23:59:59
Total running time: 5.20 mn
your time series has 1 seconds between two timestamps


  0%|                                                                                            | 0/8 [14:44<?, ?it/s]


MemoryError: Unable to allocate 3.60 GiB for an array with shape (31, 15601980) and data type object

In [8]:
#Concatenate all HenVariables csv into one
li_paths_var = glob.glob(os.path.join(path_extracted_data, id_run+'_daily__pen'+'*_variables.csv'))
li_df = []
if len(li_paths_var)!=8:
    print('ERROR: not the correct number of files, there must have one per pen')
    sys.exit()
for path_var in tqdm.tqdm(li_paths_var):
    #for being more reproductible, we open the file that was saved from cleaning
    df_ = pd.read_csv(path_var, sep=';',parse_dates=['level', 'FirstTimestamp_1_Zone', 'FirstTimestamp_2_Zone', 
                                                     'FirstTimestamp_3_Zone','FirstTimestamp_4_Zone',
                                                     'FirstTimestamp_5_Zone']) 
    print(df_.shape)
    li_df.append(df_)
df_daily = pd.concat(li_df)
df_daily.to_csv(os.path.join(path_extracted_data, id_run+'_daily_ALL_variables.csv'), sep=';', index=False)
print(df_daily.shape)
df_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
 12%|██████████▌                                                                         | 1/8 [00:05<00:39,  5.59s/it]

(5479, 165)


 25%|█████████████████████                                                               | 2/8 [00:11<00:34,  5.73s/it]

(5469, 165)


 38%|███████████████████████████████▌                                                    | 3/8 [00:18<00:29,  5.96s/it]

...

(5383, 165)


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:43<00:00,  5.47s/it]

(5508, 165)


(43289, 165)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,verification_daily_total_nbr_hour,dur_values,...,nbr_transition_at_h21,nbr_transition_at_h22,nbr_transition_at_h23,nbr_transition_next1hafterlightoff,nbr_h_per_day,correct_amount_of_hour,DOA,WOA,DIB,WIB
0,hen_130,2020-10-05,0.0,0.0,0.0,0.0,32400.0,32400.0,9.0,"[0.0, 0.0, 0.0, 0.0, 32400.0]",...,0.0,0.0,0.0,0.0,9,True,124,18,6,1
1,hen_130,2020-10-06,0.0,0.0,780.0,325.0,31295.0,32400.0,9.0,"[0.0, 0.0, 780.0, 325.0, 31295.0]",...,0.0,0.0,0.0,0.0,9,True,125,18,7,1
2,hen_130,2020-10-07,0.0,206.0,249.0,130.0,31815.0,32400.0,9.0,"[0.0, 206.0, 249.0, 130.0, 31815.0]",...,0.0,0.0,0.0,0.0,9,True,126,18,8,2


In [9]:
df_daily['level'].min()

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Timestamp('2020-09-29 00:00:00')

# Chi2distance

#computed here as its a variable and not a visual computed out of variable
#compute for every level a symmetric chi2distance heatmap
if compute_chi2_distance:   

    #sort by lowest entropy (i.e. need less info to predict futur. more predictibale should induce more similarities 
    #(as less different solution)), to potentialy make a nicer visual
    df_ = df_daily.groupby(['HenID'])['distribution_entropy'].agg(lambda x:np.mean(x)).reset_index().sort_values(['distribution_entropy'])
    li = df_['HenID'].tolist()
    axis_label = [i.split('_')[1] for i in li]
    #create path to save visual if not existing
    path_ = os.path.join(path_extracted_data,'visual','chi2distance')
    #create a director if not existing
    if not os.path.exists(path_):
        os.makedirs(path_)

    print('----------------- Compute Chi2 distance....')
    for d in li_date:
        M = np.zeros(shape=(len(li),len(li)))
        for i, h1 in enumerate(li[:-1]):
            for j in range(i+1,len(li)):
                h2 = li[j]
                li_hen_in = df_daily[df_daily['level']==d]['HenID'].unique()
                #if both hen have at least one record this day (typically not always all hen have values the first day of session)
                if (h1 in li_hen_in) & (h2 in li_hen_in):
                    l1 = df_daily[(df_daily['HenID']==h1)&(df_daily['level']==d)]['dur_values_normalized'].values[0]
                    l2 = df_daily[(df_daily['HenID']==h2)&(df_daily['level']==d)]['dur_values_normalized'].values[0]
                    chi2 = chi2_distance(l1,l2)
                    M[i][j] = chi2
                    M[j][i] = chi2
        #clear old plot
        plt.figure()
        sns.set(font_scale=0.25) 
        sns.heatmap(M, cmap="YlGnBu", xticklabels=axis_label, yticklabels=axis_label)
        plt.title(d)
        if save:
            plt.savefig(os.path.join(path_,id_run+'_chi2distance_'+str(d).split('T')[0]+'_daily_'+'.png'), dpi=300,
                        format='png',bbox_inches='tight')
        #plt.show() 
        plt.close()

